https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

input shape for VGG16
https://stackoverflow.com/questions/41903051/change-input-tensor-shape-for-vgg16-application

https://ai.stackexchange.com/questions/2928/keras-valueerror-error-when-checking-model-target-expected-activation-4-to hav

ResourceExhaustedError
https://stackoverflow.com/questions/42315289/resource-exhausted-memory-error-when-trying-to-train-a-keras-model


In [1]:
from keras.applications import VGG16

image_size = 224
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

Using TensorFlow backend.


In [2]:
# Freeze the layers except the last k layers
k = 4

for layer in vgg_conv.layers[:-k]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x7f0c714bd748> False
<keras.layers.convolutional.Conv2D object at 0x7f0be6bd72b0> False
<keras.layers.convolutional.Conv2D object at 0x7f0be6bf1198> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0be6bf1c18> False
<keras.layers.convolutional.Conv2D object at 0x7f0be63a5240> False
<keras.layers.convolutional.Conv2D object at 0x7f0be6349cf8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0be63632e8> False
<keras.layers.convolutional.Conv2D object at 0x7f0be637e128> False
<keras.layers.convolutional.Conv2D object at 0x7f0be637ed68> False
<keras.layers.convolutional.Conv2D object at 0x7f0be631b3c8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0be633bba8> False
<keras.layers.convolutional.Conv2D object at 0x7f0be62ef128> False
<keras.layers.convolutional.Conv2D object at 0x7f0be62efd68> False
<keras.layers.convolutional.Conv2D object at 0x7f0be628f3c8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0be62acba8> Fa

In [13]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 2050      
Total params: 40,407,874
Trainable params: 32,772,610
Non-trainable params: 7,635,264
_________________________________________________________________


In [16]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = '../Data/hymenoptera_data/train/'
validation_dir = '../Data/hymenoptera_data/val'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 244 images belonging to 2 classes.
Found 153 images belonging to 2 classes.


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/30
13/12 [===============================] - 8s 635ms/step - loss: 4.5990 - acc: 0.6468 - val_loss: 0.5359 - val_acc: 0.7843
Epoch 2/30
13/12 [===============================] - 5s 367ms/step - loss: 0.3896 - acc: 0.8196 - val_loss: 0.4195 - val_acc: 0.8497
Epoch 3/30
13/12 [===============================] - 5s 403ms/step - loss: 0.2741 - acc: 0.8686 - val_loss: 0.5339 - val_acc: 0.8301
Epoch 4/30
13/12 [===============================] - 5s 371ms/step - loss: 0.2770 - acc: 0.8980 - val_loss: 0.5477 - val_acc: 0.8105
Epoch 5/30
13/12 [===============================] - 5s 369ms/step - loss: 0.2466 - acc: 0.8968 - val_loss: 0.3516 - val_acc: 0.8824
Epoch 6/30
13/12 [===============================] - 5s 370ms/step - loss: 0.1862 - acc: 0.9382 - val_loss: 0.6090 - val_acc: 0.8235
Epoch 7/30
13/12 [===============================] - 5s 382ms/step - loss: 0.1488 - acc: 0.9459 - val_loss: 0.5185 - val_acc: 0.7908
Epoch 8/30
13/12 [===============================] - 5s 377ms/step - 